In [76]:
from bs4 import BeautifulSoup
import requests
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

In [77]:
r = requests.get('https://bola.kompas.com/')  #('https://www.wikipedia.org/')

In [78]:
soup = BeautifulSoup(r.content,'html.parser')

In [79]:
#Retrieve all popular new links
link = []
for i in soup.find('div', {'class':'most__wrap'}).find_all('a'):
    i['href'] = i['href'] + '?page=all'
    link.append(i['href'])

In [80]:
#Retrieve paragrahs from each link, combine each paragrah as a string and save it to docs
documents = []
for i in link:
    r = requests.get(i)
    soup = BeautifulSoup(r.content,'html.parser')
    
    sen = []
    for i in soup.find('div', {'class':'read__content'}).find_all('p'):
        sen.append(i.text)
        
    documents.append(' '.join(sen))
    

In [81]:
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

In [82]:
# Instantiate a TfIdfVectorizer Object and transform the data to vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents_clean)
X = X.T.toarray()

df = pd.DataFrame(X, index=vectorizer.get_feature_names_out())

print(df)

               0         1         2         3         4         5         6  \
ac      0.000000  0.000000  0.047832  0.000000  0.000000  0.000000  0.000000   
ada     0.000000  0.000000  0.037205  0.038026  0.024891  0.000000  0.000000   
adalah  0.028404  0.013413  0.000000  0.025528  0.000000  0.020162  0.025342   
adanya  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
adapun  0.000000  0.000000  0.037205  0.076052  0.000000  0.000000  0.037748   
...          ...       ...       ...       ...       ...       ...       ...   
yg      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
yilyu   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.085632   
yue     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.028544   
zhe     0.000000  0.025686  0.000000  0.000000  0.000000  0.038610  0.000000   
zona    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

               7         8         9  


In [83]:
#Similar Articles
def get_similar_articles(q,df):
    print("query: ",q)
    
    q = [q]
    q_vector = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    sim = {}
    
    for i in range(10):
        sim[i] = np.dot(df.loc[:, i].values, q_vector) / np.linalg.norm(df.loc[:,i]) * np.linalg.norm(q_vector)
        
    sim_sorted = sorted(sim.items(), key = lambda x:x[1], reverse = True)
    
    for k,v in sim_sorted:
        if v != 0.0:
            print("Similarities:", v)
            print(documents_clean[k])        

In [84]:
q1 = 'barcelona'

In [85]:
get_similar_articles(q1,df)

query:  barcelona
Similarities: 0.17252397014919
 kompas com hasil undian atau drawing babak playoff liga europa mempertemukan barcelona vs manchester united kepastian itu diketahui setelah drawing playoff liga europa digelar di kantor uefa nyon swiss senin malam wib drawing playoff liga europa diikuti oleh tim yang terdiri dari runner up grup liga europa dan tim peringkat ketiga fase grup liga champions kedelapan runner up grup liga europa yakni manchester united midtjylland monaco nantes psv eindhoven rennes as roma dan union berlin baca juga ada man united barcelona sampai juventus liga europa rasa liga champions adapun tim peringkat ketiga grup liga champions adalah ajax barcelona juventus bayer leverkusen salzburg sevilla shakhtar donetsk dan sporting cp sesuai dengan regulasi uefa klub dari negara yang sama tidak akan bertemu pada babak playoff liga europa adapun pemenang di babak playoff ini berhak melaju ke besar dan akan diundi dengan tim tim juara grup liga europa termasuk ar